### Realizar regresión Logistica, random forest, extra* -> red neuronal-> traducir textos a español.[REPETIR EN ESPAÑOL.]

> [ HIPÓTESIS ] La traducción afecta al rendimiento del clasificador?

Usar modelo transformer -> ingles a español **Tutorial 7**.


## 8 de enero!

### Basado en Automated Hate Speech Detection and the Problem of Offensive Language

[https://data.world/thomasrdavidson/hate-speech-and-offensive-language](https://data.world/thomasrdavidson/hate-speech-and-offensive-language)

Disclaimer WARNING: The data, lexicons, and notebooks all contain content that is racist, sexist, homophobic, and offensive in many other ways.

### Data Guide

The data are stored as a CSV, each data file contains 5 columns:

**count** = number of CrowdFlower users who coded each tweet (min is 3, sometimes more users coded a tweet when judgments were determined to be unreliable by CF).

**hate_speech** = number of CF users who judged the tweet to be hate speech.

**offensive_language** = number of CF users who judged the tweet to be offensive.

**neither** = number of CF users who judged the tweet to be neither offensive nor non-offensive.

**class** = class label for majority of CF users.
0 - hate speech,
1 - offensive language,
2 - neither



In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#abrir el dataset
d = pd.read_csv("hate1.csv",header=None)
#Eliminamos 1° columna y fila innecesarias
if len(d.columns)> 6:
    d = d.drop(d.columns[0], 1)
    d = d.iloc[1:]
d.to_csv('hate1_proc.csv')
d.head()

,1,2,3,4,5,6
1,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
2,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
3,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
4,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
5,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [3]:
d.info() # database no nulo y balanceado, en primera instancia.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24783 entries, 1 to 24783
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   1       24783 non-null  object
 1   2       24783 non-null  object
 2   3       24783 non-null  object
 3   4       24783 non-null  object
 4   5       24783 non-null  object
 5   6       24783 non-null  object
dtypes: object(6)
memory usage: 1.1+ MB


#### Cambiamos de posición la columna que clasifica el mensaje al final y reetiquetamos.

In [4]:
d = d[[1,2,3,4,6,5]]
d.columns=['count', 'hate_speech', 'offensive_language', 'neiter', 'message', 'class']
d

,count,hate_speech,offensive_language,neiter,message,class
1,3,0,0,3,!!! RT @mayasolovely: As a woman you shouldn't...,2
2,3,0,3,0,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,1
3,3,0,3,0,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,1
4,3,0,2,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,1
5,6,0,6,0,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,1
...,...,...,...,...,...,...
24779,3,0,2,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...,1
24780,3,0,1,2,"you've gone and broke the wrong heart baby, an...",2
24781,3,0,3,0,young buck wanna eat!!.. dat nigguh like I ain...,1
24782,6,0,6,0,youu got wild bitches tellin you lies,1


#### Contamos las clases

In [5]:
count_classes = d['class'].value_counts()
count_classes

1    19190
2     4163
0     1430
Name: class, dtype: int64

# Análisis de sentimientos

In [6]:
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline

import spacy
import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

### Preprocesamiento léxico  con N-gramas (estadístico)

> **spacy_tokenizer** generará un vector de palabras dividido

> **bow_vector** vectorizará las palabras de a dos (N-gramos) según su relevancia

> **tfidf_vector** transforma los textos en vectores para tener los pesos TF-IDF de cada palabra en cada documento

In [7]:
# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
stop_words=""

# Load English tokenizer, tagger, parser, NER and word vectors
parser = English()

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = parser(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

In [8]:
bow_vector = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1))
bow_vector

CountVectorizer(tokenizer=<function spacy_tokenizer at 0x7f6830ee8670>)

In [9]:
tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer)

### Partición de dataset

In [10]:
X = d['message'] # tweets 
y = d['class']   # clasificación
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .45, random_state=25)

# Entrenamiento de dataset


## Regresión Logística

In [11]:

# Logistic Regression Classifier
from sklearn.linear_model import LogisticRegression
modelLR = LogisticRegression(solver='lbfgs', max_iter=1000) # parámetros para evitar errores de iteración

# Create pipeline using Bag of Words
pipe = Pipeline([('preprocessing', bow_vector),
                 ('regression-ML', modelLR)])

# model generation
pipe.fit(X_train,y_train)

Pipeline(steps=[('preprocessing',
                 CountVectorizer(tokenizer=<function spacy_tokenizer at 0x7f6830ee8670>)),
                ('regression-ML', LogisticRegression(max_iter=1000))])

#### Evaluación del modelo

Podemos evaluar el rendimiento de nuestro modelo usando el módulo de métricas de scikit-learn. Ahora que hemos entrenado nuestro modelo, pondremos nuestros datos de prueba a disposición para hacer predicciones. Luego usaremos varias funciones del módulo de métricas para ver la exactitud, precisión y recall de nuestro modelo.

In [12]:

from sklearn import metrics
# Predicting with a test dataset
predicted = pipe.predict(X_test)
print(predicted)

# Model Accuracy
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted, average='micro'))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted, average='micro'))

['1' '2' '2' ... '2' '1' '1']
Logistic Regression Accuracy: 0.8945575181565498
Logistic Regression Precision: 0.8945575181565498
Logistic Regression Recall: 0.8945575181565498


In [13]:
#Evaluación del rendimiento del clasificador
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, predicted)
print(confusion_matrix)
#Print de la matriz de confusión
from sklearn.metrics import classification_report
print(classification_report(y_test, predicted))

[[ 137  413   64]
 [ 143 8257  258]
 [  20  278 1583]]
              precision    recall  f1-score   support

           0       0.46      0.22      0.30       614
           1       0.92      0.95      0.94      8658
           2       0.83      0.84      0.84      1881

    accuracy                           0.89     11153
   macro avg       0.74      0.67      0.69     11153
weighted avg       0.88      0.89      0.89     11153



In [14]:

def printNMostInformative(vectorizer, model, N):
    feature_names = vectorizer.get_feature_names()
    coefs_with_fns = sorted(zip(model.coef_[0], feature_names))
    topClass1 = coefs_with_fns[:N]
    topClass2 = coefs_with_fns[:-(N + 1):-1]
    print("Class 1 best: ")
    for feat in topClass1:
        print(feat)
    print("Class 2 best: ")
    for feat in topClass2:
        print(feat)

In [15]:

printNMostInformative(bow_vector, modelLR, 20)

Class 1 best: 
(-1.2172576985379937, 'bird')
(-0.9325509360293304, 'imma')
(-0.813715662921115, 'remember')
(-0.7968451014006322, 'ho')
(-0.7921762177220638, 'birds')
(-0.7896245992880705, 'charlie')
(-0.7642825242895863, 'crazy')
(-0.7459865448355021, 'its')
(-0.6921438986726948, 'yellow')
(-0.6878548898299696, 'weed')
(-0.6743548047900063, 'woman')
(-0.6622771136407989, 'girls')
(-0.6438012085947404, '@ivanrabago')
(-0.6429506789381668, 'til')
(-0.6415810436620708, 'love')
(-0.6405212992654807, 'red')
(-0.6335319750614081, 'after')
(-0.6314268450587154, 'hard')
(-0.6309556853832609, 'great')
(-0.6185010080275121, 'single')
Class 2 best: 
(2.4334250390032315, 'faggot')
(2.238845917901955, 'nigger')
(2.170525625455344, 'fag')
(2.1503749693819487, 'niggers')
(2.0609249182638956, 'nigga')
(1.9196117835156117, 'faggots')
(1.8071335056339664, 'fags')
(1.7293837046719647, 'dyke')
(1.5888244033064869, 'fuck')
(1.5737963098636247, 'queer')
(1.5713616724127986, 'white')
(1.5448913886110813, 'c

### Limitaciones de los modelos de lenguaje N-Grama

1) Un modelo de lenguaje N-Grama con un valor N mayor es más preciso pero genera problemas de computación.

2) Los modelos N-gramas son representaciones escasa/ingenua del lenguaje. Solo consideran la forma de las palabras y no su significado/semántica

Para mejorar estas limitaciones:

Word Embedding (proyección semántica de las palabras a través de vectores): Word2Vec, GLoVe

Modelos de lenguaje neuronales: BERT, GPT-2, GPT-3

# Word Embedding

### Def:
El concepto de word embedding se refiere a un conjunto de técnicas utilizadas para aprender representaciones matemáticas, tipicamente vectores, de cada palabra.

Una de las técnicas más populares es Word2Vec propuesto por un equipo de investigación de Google en 2013 (Efficient Estimation of Word Representations in Vector Space [Mikolov et al., 2013]).

Alternativas populares son GloVe (propuesta por la Universidad de Stanford en 2014) y FastText (propuesta por Facebook en 2016), que extende Word2Vec para considerar de mejor manera las palabras con errores ortográficas.

Se empleará el algoritmo **CBOW** para vectorizar las probabilidades de los textos de pertenecer a ciertas clases de palabras, empleando **word2vec** de Gensim

In [16]:
X = d['message']
X

1        !!! RT @mayasolovely: As a woman you shouldn't...
2        !!!!! RT @mleew17: boy dats cold...tyga dwn ba...
3        !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
4        !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
5        !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
                               ...                        
24779    you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24780    you've gone and broke the wrong heart baby, an...
24781    young buck wanna eat!!.. dat nigguh like I ain...
24782                youu got wild bitches tellin you lies
24783    ~~Ruffled | Ntac Eileen Dahlia - Beautiful col...
Name: message, Length: 24783, dtype: object

In [17]:
from gensim.models import word2vec
#sentences = word2vec.Text8Corpus(X) 

In [18]:
#!pip install torch==1.5.1
#!pip install torchtext==0.6


In [19]:
import torch
import spacy
import random
import torchtext
from torchtext import data
from torchtext import datasets
spacy.load('en_core_web_sm')

In [20]:
#!python -m spacy download en
TEXT = data.Field(tokenize='spacy', batch_first = True)
TOXIC = data.LabelField(dtype = torch.float)

In [22]:

fields = [(None, None),(None, None),(None, None),(None, None),('toxic_level', TOXIC), ('message', TEXT), (None, None) ]

In [23]:
import pandas as pd 
rows = pd.read_csv("hate1_proc.csv", chunksize=int(len(X)/(2.5))) 
for i, chuck in enumerate(rows): 
    chuck.to_csv('out{}.csv'.format(i)) # i is for chunk number of each iteration 

In [24]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

train_data, valid_data, test_data = data.TabularDataset.splits(
                                        path = '.',
                                        train = 'out0.csv',
                                        validation= 'out1.csv',
                                        test = 'out2.csv',
                                        format = 'csv',
                                        fields = fields,
                                        skip_header = True
)

In [30]:

BATCH_SIZE = 32

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device,
    sort_key=lambda x:len(x.toxicity),
    sort_within_batch=False)

In [25]:
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

TOXIC.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [43:46, 328kB/s]                                
100%|█████████▉| 399999/400000 [00:15<00:00, 25065.67it/s]


In [26]:
import torch.nn as nn
import torch.nn.functional as F

class CNN1d(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.convs = nn.ModuleList([
                                    nn.Conv1d(in_channels = embedding_dim, 
                                              out_channels = n_filters, 
                                              kernel_size = fs)
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [batch size, sent len]
        
        embedded = self.embedding(text)
                
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.permute(0, 2, 1)
        
        #embedded = [batch size, emb dim, sent len]
        
        conved = [F.relu(conv(embedded)) for conv in self.convs]
            
        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]
        
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim = 1))
        
        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)

In [27]:

INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [3]
OUTPUT_DIM = 1
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = CNN1d(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)

In [28]:
pretrained_embeddings = TEXT.vocab.vectors
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data.copy_(pretrained_embeddings)
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

In [31]:
#OPTIMIZAR EL MODELO
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [37]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.message).squeeze(1)
        
        loss = criterion(predictions, batch.toxic_level)
        
        acc = binary_accuracy(predictions, batch.toxicity)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [33]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [34]:

def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [35]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:
           
            predictions = model(batch.comment_text).squeeze(1)
            
            loss = criterion(predictions, batch.toxicity)
            
            acc = binary_accuracy(predictions, batch.toxicity)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [38]:

N_EPOCHS = 2

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        nombre = './toxic-model-CNN'+'_ep'+str(epoch+1)+'.pt'
        torch.save({'epoca': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'Valid_loss': best_valid_loss}, nombre)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

RuntimeError: Calculated padded input size per channel: (1). Kernel size: (3). Kernel size can't be greater than actual input size

In [ ]:

best_model = CNN1d(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)

In [ ]:
pretrained_embeddings = TEXT.vocab.vectors
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

best_model.embedding.weight.data.copy_(pretrained_embeddings)
best_model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
best_model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

In [ ]:

name = './toxic-model-CNN'+'_ep'+str(2)+'.pt'
best_model.load_state_dict(torch.load(name)['model_state_dict'])

In [ ]:
from sklearn.metrics import f1_score,confusion_matrix, classification_report

In [ ]:
prediction_test = []
labels_test=[]
for batch in test_iterator:
    labels_test.append(batch.toxicity.cpu().detach().numpy())
    predictions = best_model(batch.comment_text.cpu()).squeeze(1)
    rounded_preds = torch.round(torch.sigmoid(predictions))
    prediction_test.append(rounded_preds.detach().numpy())
    

y_true = np.concatenate(labels_test)
y_pred = np.concatenate(prediction_test)

In [ ]:
display(y_pred,y_true)

In [ ]:

cm = confusion_matrix(y_true, y_pred)
display(cm)

print(classification_report(y_true, y_pred))

In [ ]:
# jUGAR CON EL MODELO

In [ ]:
import spacy
nlp = spacy.load('en')

def predict_toxicity(model, sentence, min_len = 5):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    if len(tokenized) < min_len:
        tokenized += ['<pad>'] * (min_len - len(tokenized))
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [ ]:
predict_toxicity(best_model, "You are very stupid!")